In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = pd.read_csv('./finalData/top_10_tags_encoded.csv', encoding='latin1')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepLearning/project/top_10_tags_encoded.csv', encoding='latin1')

Tokenization & Padding


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Parameters
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 300

# Tokenize
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])

# Pad
X_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Labels (already one-hot encoded)
y = df.iloc[:, 1:].values  # assuming clean_text is the first column

Train-Test Split


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_seq, y, test_size=0.2, random_state=42)

LSTM Model


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=128))  # Removed input_length
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='sigmoid'))  # 10 tags (multi-label)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Train the Model


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_data=(X_val, y_val))

Epoch 1/5
4411/4411 ━━━━━━━━━━━━━━━━━━━━ 2940s 665ms/step - accuracy: 0.6014 - loss: 0.1924 - val_accuracy: 0.8385 - val_loss: 0.0789
Epoch 2/5
4411/4411 ━━━━━━━━━━━━━━━━━━━━ 2907s 653ms/step - accuracy: 0.8433 - loss: 0.0768 - val_accuracy: 0.8426 - val_loss: 0.0748
Epoch 3/5
4411/4411 ━━━━━━━━━━━━━━━━━━━━ 2964s 667ms/step - accuracy: 0.8499 - loss: 0.0714 - val_accuracy: 0.8453 - val_loss: 0.0741
Epoch 4/5
4411/4411 ━━━━━━━━━━━━━━━━━━━━ 2939s 662ms/step - accuracy: 0.8546 - loss: 0.0681 - val_accuracy: 0.8452 - val_loss: 0.0739
Epoch 5/5
4411/4411 ━━━━━━━━━━━━━━━━━━━━ 2887s 655ms/step - accuracy: 0.8576 - loss: 0.0657 - val_accuracy: 0.8431 - val_loss: 0.0740


Save the model

In [ ]:
model.save("lstm_multilabel_model.keras")

In [ ]:
from google.colab import files
files.download("lstm_multilabel_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>